In [311]:
import sys
from pathlib import Path

# add project root to Python path
PROJECT_ROOT = Path.cwd().parents[0]
sys.path.append(str(PROJECT_ROOT))

print("Project root added to sys.path:", PROJECT_ROOT)


Project root added to sys.path: c:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report


In [312]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from linearmodels.system import SUR
from statsmodels.stats.outliers_influence import variance_inflation_factor
from src.paths import RAW_DATA_DIR, PROCESSED_DATA_DIR, OUTPUTS_DIR, TABLES_DIR, FIGURES_DIR



In [313]:
base_file = RAW_DATA_DIR / "NHTS_2017"

In [314]:
nhts_2017_person = base_file / "perpub.csv"
nhts_2017_trip   = base_file / "trippub.csv"
nhts_2017_HH     = base_file / "hhpub.csv"

In [315]:
nhts2017_trip_1 = pd.read_csv(nhts_2017_trip)


In [316]:
nhts2017_HH_1 = pd.read_csv(nhts_2017_HH)
nhts2017_HH_1

,HOUSEID,TRAVDAY,SAMPSTRAT,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,PC,SPHONE,TAB,...,SMPLSRCE,WTHHFIN,HBHUR,HTHTNRNT,HTPPOPDN,HTRESDN,HTEEMPDN,HBHTNRNT,HBPPOPDN,HBRESDN
0,30000007,2,3,1,3,5,7,2,1,2,...,2,187.314320,T,50,1500,750,750,20,750,300
1,30000008,5,2,1,2,4,8,1,1,2,...,2,69.513032,R,5,300,300,150,5,300,300
2,30000012,5,3,1,1,2,10,1,1,3,...,2,79.419586,C,80,17000,17000,5000,60,17000,7000
3,30000019,5,3,1,2,2,3,1,5,5,...,2,279.143588,S,40,300,300,150,50,750,300
4,30000029,3,3,1,2,2,5,2,5,1,...,2,103.240304,S,40,1500,750,750,40,1500,750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129691,40794291,7,3,1,1,1,9,1,5,5,...,2,41.869638,S,5,750,300,150,20,750,300
129692,40794293,2,4,1,2,3,6,1,1,1,...,2,166.016235,R,20,50,50,25,20,50,50
129693,40794294,5,2,1,2,2,10,1,1,5,...,2,207.672765,U,70,7000,7000,5000,70,17000,7000
129694,40794301,2,3,1,7,5,6,1,5,1,...,2,126.720282,R,20,50,50,25,20,300,50


In [317]:
HH_col = ["HOUSEID", "YOUNGCHILD"]
nhts2017_HH = nhts2017_HH_1[HH_col]
nhts2017_HH

,HOUSEID,YOUNGCHILD
0,30000007,0
1,30000008,0
2,30000012,0
3,30000019,0
4,30000029,0
...,...,...
129691,40794291,0
129692,40794293,0
129693,40794294,0
129694,40794301,0


In [318]:
nhts2017_per_1 = pd.read_csv(nhts_2017_person)
nhts2017_per_1

,HOUSEID,PERSONID,R_AGE,EDUC,R_HISP,R_RELAT,R_SEX,R_RACE,PRMACT,PAYPROF,...,SMPLSRCE,WTPERFIN,HBHUR,HTHTNRNT,HTPPOPDN,HTRESDN,HTEEMPDN,HBHTNRNT,HBPPOPDN,HBRESDN
0,30000007,1,67,3,2,1,2,2,6,2,...,2,206.690153,T,50,1500,750,750,20,750,300
1,30000007,2,66,3,2,2,1,2,1,-1,...,2,197.075742,T,50,1500,750,750,20,750,300
2,30000007,3,28,2,2,3,2,2,5,2,...,2,219.514210,T,50,1500,750,750,20,750,300
3,30000008,1,55,5,2,1,1,1,1,-1,...,2,63.185911,R,5,300,300,150,5,300,300
4,30000008,2,49,4,2,2,2,1,1,-1,...,2,58.665911,R,5,300,300,150,5,300,300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264229,40794301,4,18,2,2,3,2,1,97,2,...,2,213.514657,R,20,50,50,25,20,300,50
264230,40794301,5,14,1,2,3,2,1,-1,-1,...,2,133.448420,R,20,50,50,25,20,300,50
264231,40794301,6,12,-1,2,3,2,1,-1,-1,...,2,133.448420,R,20,50,50,25,20,300,50
264232,40794301,7,8,-1,2,3,2,1,-1,-1,...,2,133.448420,R,20,50,50,25,20,300,50


## Deal with Trip

In [319]:
#only trip
columns_trip_2017 = [
    "HOUSEID", "PERSONID", 
    "TRVLCMIN", "DWELTIME", "WHYTO", 
]
nhts2017_trip = nhts2017_trip_1[columns_trip_2017]

In [320]:
nhts2017_trip["uni_Per_ID"]=nhts2017_trip["HOUSEID"].astype(str) + "_" + nhts2017_trip["PERSONID"].astype(str)
print(nhts2017_trip.shape)

(923572, 6)


C:\Users\jinzh\AppData\Local\Temp\ipykernel_36784\808356664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2017_trip["uni_Per_ID"]=nhts2017_trip["HOUSEID"].astype(str) + "_" + nhts2017_trip["PERSONID"].astype(str)


In [321]:
#nhts2017_trip = nhts2017_trip.replace([-9, -7, -8,-6,-5,-4,-3,-2,-1], 0)

In [322]:
nhts2017_trip

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,uni_Per_ID
0,30000007,1,15,295,19,30000007_1
1,30000007,1,20,-9,1,30000007_1
2,30000007,2,120,540,1,30000007_2
3,30000007,2,150,-9,3,30000007_2
4,30000007,3,15,330,8,30000007_3
...,...,...,...,...,...,...
923567,50515573,1,27,283,3,50515573_1
923568,50515573,1,8,47,16,50515573_1
923569,50515573,1,5,240,3,50515573_1
923570,50515573,1,25,3,12,50515573_1


In [323]:
valid_ids = nhts2017_trip.loc[
    (nhts2017_trip["DWELTIME"] == -9) & (nhts2017_trip["WHYTO"].isin([1,2])),
    "uni_Per_ID"
].unique()

filtered_nhts2017_trip = nhts2017_trip[nhts2017_trip["uni_Per_ID"].isin(valid_ids)]


In [324]:
filtered_nhts2017_trip 

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,uni_Per_ID
0,30000007,1,15,295,19,30000007_1
1,30000007,1,20,-9,1,30000007_1
4,30000007,3,15,330,8,30000007_3
5,30000007,3,15,-9,1,30000007_3
6,30000008,1,15,720,3,30000008_1
...,...,...,...,...,...,...
923567,50515573,1,27,283,3,50515573_1
923568,50515573,1,8,47,16,50515573_1
923569,50515573,1,5,240,3,50515573_1
923570,50515573,1,25,3,12,50515573_1


In [325]:
filtered_nhts2017_trip.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO
count,8.731230e+05,873123.000000,873123.000000,873123.000000,873123.000000
mean,3.490963e+07,1.660782,20.683171,90.585648,7.342261
std,5.044677e+06,0.936607,29.554877,150.063120,7.693029
min,3.000001e+07,1.000000,-9.000000,-9.000000,-9.000000
25%,3.025597e+07,1.000000,8.000000,1.000000,1.000000
50%,3.051310e+07,1.000000,15.000000,28.000000,6.000000
75%,4.035721e+07,2.000000,25.000000,105.000000,12.000000
max,5.051557e+07,11.000000,1200.000000,1331.000000,97.000000


In [326]:
filtered_nhts2017_trip["uni_Per_ID"].nunique()

204871

In [327]:
nhts2017_trip = filtered_nhts2017_trip.replace([-9, -7, -8,-6,-5,-4,-3,-2,-1], 0)

In [328]:
nhts2017_trip.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO
count,8.731230e+05,873123.000000,873123.000000,873123.000000,873123.000000
mean,3.490963e+07,1.660782,20.694550,92.697485,7.342909
std,5.044677e+06,0.936607,29.545177,148.703964,7.692091
min,3.000001e+07,1.000000,0.000000,0.000000,0.000000
25%,3.025597e+07,1.000000,8.000000,1.000000,1.000000
50%,3.051310e+07,1.000000,15.000000,28.000000,6.000000
75%,4.035721e+07,2.000000,25.000000,105.000000,12.000000
max,5.051557e+07,11.000000,1200.000000,1331.000000,97.000000


In [329]:
# Regular home activities (chores, sleep)
nhts2017_trip["AT_REGUHM"] = np.where(
    nhts2017_trip["WHYTO"] == 1,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_REGUHM"] = np.where(
    nhts2017_trip["WHYTO"] == 1,
    nhts2017_trip["TRVLCMIN"],
    0
)

# Work from home (paid)
nhts2017_trip["AT_WKFHMP"] = np.where(
    nhts2017_trip["WHYTO"] == 2,
    nhts2017_trip["DWELTIME"],
    0
)
nhts2017_trip["TT_WKFHMP"] = np.where(
    nhts2017_trip["WHYTO"] == 2,
    nhts2017_trip["TRVLCMIN"],
    0
)

# Work
nhts2017_trip["AT_WK"] = np.where(
    nhts2017_trip["WHYTO"] == 3,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_WK"] = np.where(
    nhts2017_trip["WHYTO"] == 3,
    nhts2017_trip["TRVLCMIN"],
    0
)

# Work-related meeting / trip
nhts2017_trip["AT_WKRMEET"] = np.where(
    nhts2017_trip["WHYTO"] == 4,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_WKRMEET"] = np.where(
    nhts2017_trip["WHYTO"] == 4,
    nhts2017_trip["TRVLCMIN"],
    0
)

# Volunteer activities (not paid)
nhts2017_trip["AT_VOACT"] = np.where(
    nhts2017_trip["WHYTO"] == 5,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_VOACT"] = np.where(
    nhts2017_trip["WHYTO"] == 5,
    nhts2017_trip["TRVLCMIN"],
    0
)

# Drop off /pick up someone
nhts2017_trip["AT_DOFPU"] = np.where(
    nhts2017_trip["WHYTO"] == 6,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_DOFPU"] = np.where(
    nhts2017_trip["WHYTO"] == 6,
    nhts2017_trip["TRVLCMIN"],
    0
)

# Change type of transportation
nhts2017_trip["AT_CTYTRANS"] = np.where(
    nhts2017_trip["WHYTO"] == 7,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_CTYTRANS"] = np.where(
    nhts2017_trip["WHYTO"] == 7,
    nhts2017_trip["TRVLCMIN"],
    0
)

# Attend school as a student
nhts2017_trip["AT_SCHSTU"] = np.where(
    nhts2017_trip["WHYTO"] == 8,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_SCHSTU"] = np.where(
    nhts2017_trip["WHYTO"] == 8,
    nhts2017_trip["TRVLCMIN"],
    0
)

# Attend child care
nhts2017_trip["AT_CHILDC"] = np.where(
    nhts2017_trip["WHYTO"] == 9,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_CHILDC"] = np.where(
    nhts2017_trip["WHYTO"] == 9,
    nhts2017_trip["TRVLCMIN"],
    0
)

# Attend adult care
nhts2017_trip["AT_ADULTC"] = np.where(
    nhts2017_trip["WHYTO"] == 10,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_ADULTC"] = np.where(
    nhts2017_trip["WHYTO"] == 10,
    nhts2017_trip["TRVLCMIN"],
    0
)

# Buy goods (groceries, clothes, appliances, gas)
nhts2017_trip["AT_BUYGOOD"] = np.where(
    nhts2017_trip["WHYTO"] == 11,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_BUYGOOD"] = np.where(
    nhts2017_trip["WHYTO"] == 11,
    nhts2017_trip["TRVLCMIN"],
    0
)

# 12=Buy services (dry cleaners, banking, service a car, pet care)
nhts2017_trip["AT_BUYSER"] = np.where(
    nhts2017_trip["WHYTO"] == 12,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_BUYSER"] = np.where(
    nhts2017_trip["WHYTO"] == 12,
    nhts2017_trip["TRVLCMIN"],
    0
)

# Buy services (dry cleaners, banking, service a car, pet care)
nhts2017_trip["AT_BUYMEAL"] = np.where(
    nhts2017_trip["WHYTO"] == 13,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_BUYMEAL"] = np.where(
    nhts2017_trip["WHYTO"] == 13,
    nhts2017_trip["TRVLCMIN"],
    0
)

# 14. Other general errands (post office, library)
nhts2017_trip["AT_ERRAND"] = np.where(
    nhts2017_trip["WHYTO"] == 14,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_ERRAND"] = np.where(
    nhts2017_trip["WHYTO"] == 14,
    nhts2017_trip["TRVLCMIN"],
    0
)

# 15=Recreational activities (visit parks, movies, bars, museums)
nhts2017_trip["AT_RECREAT"] = np.where(
    nhts2017_trip["WHYTO"] == 15,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_RECREAT"] = np.where(
    nhts2017_trip["WHYTO"] == 15,
    nhts2017_trip["TRVLCMIN"],
    0
)

# 16=Exercise (go for a jog, walk, walk the dog, go to the gym)
nhts2017_trip["AT_EXERS"] = np.where(
    nhts2017_trip["WHYTO"] == 16,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_EXERS"] = np.where(
    nhts2017_trip["WHYTO"] == 16,
    nhts2017_trip["TRVLCMIN"],
    0
)

# 17=Visit friends or relatives
nhts2017_trip["AT_VISFRI"] = np.where(
    nhts2017_trip["WHYTO"] == 17,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_VISFRI"] = np.where(
    nhts2017_trip["WHYTO"] == 17,
    nhts2017_trip["TRVLCMIN"],
    0
)

# 18=Health care visit (medical, dental, therapy)
nhts2017_trip["AT_HEAL"] = np.where(
    nhts2017_trip["WHYTO"] == 18,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_HEAL"] = np.where(
    nhts2017_trip["WHYTO"] == 18,
    nhts2017_trip["TRVLCMIN"],
    0
)

# 19=Religious or other community activities
nhts2017_trip["AT_RELIG"] = np.where(
    nhts2017_trip["WHYTO"] == 19,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_RELIG"] = np.where(
    nhts2017_trip["WHYTO"] == 19,
    nhts2017_trip["TRVLCMIN"],
    0
)

# 97=Something else
nhts2017_trip["AT_SOMELSE"] = np.where(
    nhts2017_trip["WHYTO"] == 97,
    nhts2017_trip["DWELTIME"],
    0
)

nhts2017_trip["TT_SOMELSE"] = np.where(
    nhts2017_trip["WHYTO"] == 97,
    nhts2017_trip["TRVLCMIN"],
    0
)

In [330]:
nhts2017_trip.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,TT_REGUHM,AT_WKFHMP,TT_WKFHMP,AT_WK,...,AT_EXERS,TT_EXERS,AT_VISFRI,TT_VISFRI,AT_HEAL,TT_HEAL,AT_RELIG,TT_RELIG,AT_SOMELSE,TT_SOMELSE
count,8.731230e+05,873123.000000,873123.000000,873123.000000,873123.000000,873123.000000,873123.000000,873123.000000,873123.000000,873123.000000,...,873123.000000,873123.000000,873123.000000,873123.000000,873123.000000,873123.000000,873123.000000,873123.000000,873123.000000,873123.000000
mean,3.490963e+07,1.660782,20.694550,92.697485,7.342909,15.504174,7.990517,1.280534,0.138231,37.123523,...,2.300213,0.519514,4.760986,0.795171,1.410265,0.474752,2.331881,0.365024,0.142556,0.078209
std,5.044677e+06,0.936607,29.545177,148.703964,7.692091,59.457815,23.028712,21.875686,2.710096,126.739731,...,17.025790,4.570054,33.802463,7.072334,14.098978,4.981677,19.731379,3.814467,5.282301,2.397697
min,3.000001e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.025597e+07,1.000000,8.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.051310e+07,1.000000,15.000000,28.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.035721e+07,2.000000,25.000000,105.000000,12.000000,0.000000,10.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.051557e+07,11.000000,1200.000000,1331.000000,97.000000,1260.000000,1200.000000,940.000000,730.000000,1240.000000,...,1045.000000,570.000000,1110.000000,660.000000,930.000000,480.000000,1020.000000,420.000000,855.000000,450.000000


In [331]:
nhts2017_trip.columns

Index(['HOUSEID', 'PERSONID', 'TRVLCMIN', 'DWELTIME', 'WHYTO', 'uni_Per_ID',
       'AT_REGUHM', 'TT_REGUHM', 'AT_WKFHMP', 'TT_WKFHMP', 'AT_WK', 'TT_WK',
       'AT_WKRMEET', 'TT_WKRMEET', 'AT_VOACT', 'TT_VOACT', 'AT_DOFPU',
       'TT_DOFPU', 'AT_CTYTRANS', 'TT_CTYTRANS', 'AT_SCHSTU', 'TT_SCHSTU',
       'AT_CHILDC', 'TT_CHILDC', 'AT_ADULTC', 'TT_ADULTC', 'AT_BUYGOOD',
       'TT_BUYGOOD', 'AT_BUYSER', 'TT_BUYSER', 'AT_BUYMEAL', 'TT_BUYMEAL',
       'AT_ERRAND', 'TT_ERRAND', 'AT_RECREAT', 'TT_RECREAT', 'AT_EXERS',
       'TT_EXERS', 'AT_VISFRI', 'TT_VISFRI', 'AT_HEAL', 'TT_HEAL', 'AT_RELIG',
       'TT_RELIG', 'AT_SOMELSE', 'TT_SOMELSE'],
      dtype='object')

In [332]:
nhts_agg_per_2017 = (
    nhts2017_trip
    .groupby("uni_Per_ID")
    .agg({
         "HOUSEID": "first", 
         "PERSONID":"first",  
         "TRVLCMIN":"first", 
         "DWELTIME":"first", 
         "WHYTO":"first", 
        
        # Activity Time use columns               
        "AT_REGUHM": "sum",
        "AT_WKFHMP": "sum",
        "AT_WK": "sum",
        "AT_WKRMEET": "sum",
        "AT_VOACT": "sum",
        "AT_DOFPU": "sum",
        "AT_CTYTRANS": "sum",
        "AT_SCHSTU": "sum",
        "AT_CHILDC": "sum",
        "AT_ADULTC": "sum",
        "AT_BUYGOOD": "sum",
        "AT_BUYSER": "sum",
        "AT_BUYMEAL": "sum",
        "AT_ERRAND": "sum",
        "AT_RECREAT": "sum",
        "AT_EXERS": "sum",
        "AT_VISFRI": "sum",
        "AT_HEAL": "sum",
        "AT_RELIG": "sum",
        "AT_SOMELSE": "sum",

        #Trip Time Use Columns
        "TT_REGUHM": "sum",
        "TT_WKFHMP": "sum",
        "TT_WK":"sum",
        "TT_WKRMEET": "sum",
        "TT_VOACT":"sum",
        "TT_DOFPU":"sum",
        "TT_CTYTRANS": "sum",
        "TT_SCHSTU": "sum",
        "TT_CHILDC": "sum",
        "TT_ADULTC" : "sum",
        "TT_BUYGOOD": "sum",
        "TT_BUYSER": "sum",
        "TT_BUYMEAL": "sum",
        "TT_ERRAND": "sum",
        "TT_RECREAT": "sum",
        "TT_EXERS": "sum",
        "TT_VISFRI": "sum",
        "TT_HEAL": "sum",
        "TT_RELIG": "sum",
        "TT_SOMELSE": "sum"
    })
    .reset_index(drop=False)
)


In [333]:
nhts_agg_per_2017

,uni_Per_ID,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WK,AT_WKRMEET,...,TT_BUYGOOD,TT_BUYSER,TT_BUYMEAL,TT_ERRAND,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_HEAL,TT_RELIG,TT_SOMELSE
0,30000007_1,30000007,1,15,295,19,0,0,0,0,...,0,0,0,0,0,0,0,0,15,0
1,30000007_3,30000007,3,15,330,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30000008_1,30000008,1,15,720,3,0,0,720,0,...,0,0,0,0,0,0,0,0,0,0
3,30000012_1,30000012,1,15,55,16,130,0,50,295,...,0,0,0,0,0,25,95,0,0,0
4,30000019_1,30000019,1,15,105,18,0,0,0,0,...,0,0,0,0,0,0,0,15,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204866,40794293_2,40794293,2,35,505,14,0,0,0,0,...,0,0,0,35,0,0,0,0,0,0
204867,40794301_2,40794301,2,30,150,3,0,0,430,0,...,16,0,10,0,0,0,0,0,0,0
204868,40794301_3,40794301,3,26,1,6,0,0,0,0,...,7,0,0,0,0,0,0,0,0,0
204869,40794301_6,40794301,6,26,72,16,0,0,0,0,...,0,0,0,0,0,26,0,0,0,0


In [334]:
nhts_agg_per_2017.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WK,AT_WKRMEET,AT_VOACT,...,TT_BUYGOOD,TT_BUYSER,TT_BUYMEAL,TT_ERRAND,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_HEAL,TT_RELIG,TT_SOMELSE
count,2.048710e+05,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,...,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000
mean,3.499746e+07,1.759737,24.587428,192.961312,9.500066,66.075975,5.457405,158.213715,4.552401,4.886787,...,9.924411,1.697024,5.857364,1.962279,3.529436,2.214076,3.388874,2.023307,1.555667,0.333312
std,5.051721e+06,0.999996,39.022362,212.514233,7.720353,124.230255,47.405799,239.732696,35.911721,36.731736,...,24.453256,8.586645,18.659721,9.819077,16.794305,10.158935,15.340508,10.617758,8.282167,5.404368
min,3.000001e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.025987e+07,1.000000,10.000000,20.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.052146e+07,1.000000,15.000000,85.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.036688e+07,2.000000,30.000000,375.000000,14.000000,82.000000,0.000000,408.000000,0.000000,0.000000,...,12.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.051557e+07,11.000000,1200.000000,1331.000000,97.000000,1260.000000,965.000000,1240.000000,1110.000000,1331.000000,...,960.000000,800.000000,840.000000,690.000000,735.000000,735.000000,660.000000,480.000000,420.000000,450.000000


In [335]:
# drop the person that travel time is larger than 6 hour
travel_time_2017 = [
    "TT_REGUHM", "TT_WKFHMP", "TT_WK", "TT_WKRMEET", "TT_VOACT",
        "TT_DOFPU", "TT_CTYTRANS", "TT_SCHSTU", "TT_CHILDC", "TT_ADULTC",
        "TT_BUYGOOD", "TT_BUYSER", "TT_BUYMEAL", "TT_ERRAND", "TT_RECREAT",
        "TT_EXERS", "TT_VISFRI", "TT_HEAL", "TT_RELIG", "TT_SOMELSE"
]
nhts_agg_per_2017["Total_TT"] = nhts_agg_per_2017[travel_time_2017].sum(axis=1)

In [336]:
nhts_agg_per_2017["Total_TT"].describe()

count    204871.000000
mean         88.181602
std          80.451693
min           0.000000
25%          40.000000
50%          67.000000
75%         110.000000
max        1217.000000
Name: Total_TT, dtype: float64

In [337]:
regular_TT_2017 = nhts_agg_per_2017[nhts_agg_per_2017["Total_TT"] <= 360]
supercommute_TT_2017 = nhts_agg_per_2017[nhts_agg_per_2017["Total_TT"] > 360]

In [338]:
supercommute_TT_2017

,uni_Per_ID,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WK,AT_WKRMEET,...,TT_BUYSER,TT_BUYMEAL,TT_ERRAND,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_HEAL,TT_RELIG,TT_SOMELSE,Total_TT
38,30000194_3,30000194,3,395,35,11,0,0,0,0,...,0,0,0,0,0,0,0,0,0,410
50,30000289_1,30000289,1,15,5,11,265,0,420,75,...,0,0,0,0,0,0,0,0,0,450
51,30000289_2,30000289,2,10,10,11,0,0,0,0,...,0,0,0,0,0,215,0,0,0,375
52,30000289_3,30000289,3,15,10,11,540,0,0,0,...,0,0,0,0,0,0,0,0,0,665
86,30000463_1,30000463,1,14,22,1,125,0,0,241,...,19,31,0,0,0,0,0,0,0,374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204717,40793049_2,40793049,2,600,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,600
204743,40793250_1,40793250,1,120,15,11,0,0,0,0,...,0,100,0,0,0,0,0,0,0,595
204744,40793250_2,40793250,2,120,15,11,0,0,0,0,...,0,105,0,0,0,0,0,0,0,615
204766,40793520_1,40793520,1,5,45,6,0,0,0,0,...,0,0,0,220,0,0,0,0,0,490


In [339]:
AT_timeusesum_2017 = [
       "AT_REGUHM", "AT_WKFHMP", "AT_WK", "AT_WKRMEET", "AT_VOACT",
        "AT_DOFPU", "AT_CTYTRANS", "AT_SCHSTU", "AT_CHILDC", "AT_ADULTC",
        "AT_BUYGOOD", "AT_BUYSER", "AT_BUYMEAL", "AT_ERRAND", "AT_RECREAT",
        "AT_EXERS", "AT_VISFRI", "AT_HEAL", "AT_RELIG", "AT_SOMELSE",
]
# All activity time
nhts_agg_per_2017["Total_AT"] = nhts_agg_per_2017[AT_timeusesum_2017].sum(axis=1)
# Activity time + Travel Time
Totaltime2017 = ["Total_AT", "Total_TT"]
# Trip end at home, so the time at home need to be calculated
nhts_agg_per_2017["HW_TIME_2"] = 1440 - nhts_agg_per_2017[Totaltime2017].sum(axis=1)
# Time at home + regular time spend at home
nhts_agg_per_2017["HW_TIME_1"] = nhts_agg_per_2017["AT_REGUHM"] + nhts_agg_per_2017["TT_REGUHM"]+ nhts_agg_per_2017["HW_TIME_2"] + nhts_agg_per_2017["AT_WKFHMP"] + nhts_agg_per_2017["TT_WKFHMP"]


In [340]:
nhts_agg_per_2017.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WK,AT_WKRMEET,AT_VOACT,...,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_HEAL,TT_RELIG,TT_SOMELSE,Total_TT,Total_AT,HW_TIME_2,HW_TIME_1
count,2.048710e+05,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,...,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000,204871.000000
mean,3.499746e+07,1.759737,24.587428,192.961312,9.500066,66.075975,5.457405,158.213715,4.552401,4.886787,...,3.529436,2.214076,3.388874,2.023307,1.555667,0.333312,88.181602,395.029145,956.789253,1062.965881
std,5.051721e+06,0.999996,39.022362,212.514233,7.720353,124.230255,47.405799,239.732696,35.911721,36.731736,...,16.794305,10.158935,15.340508,10.617758,8.282167,5.404368,80.451693,234.710260,253.335620,239.129960
min,3.000001e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-140.000000,75.000000
25%,3.025987e+07,1.000000,10.000000,20.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.000000,180.000000,767.000000,867.000000
50%,3.052146e+07,1.000000,15.000000,85.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67.000000,431.000000,915.000000,1065.000000
75%,4.036688e+07,2.000000,30.000000,375.000000,14.000000,82.000000,0.000000,408.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,110.000000,572.000000,1170.000000,1282.000000
max,5.051557e+07,11.000000,1200.000000,1331.000000,97.000000,1260.000000,965.000000,1240.000000,1110.000000,1331.000000,...,735.000000,735.000000,660.000000,480.000000,420.000000,450.000000,1217.000000,1494.000000,1440.000000,1440.000000


In [341]:
# Remove outliers: Keep only rows with Total_time between 0 and 1440
nhts_agg_per_2017 = nhts_agg_per_2017[
    (nhts_agg_per_2017[Totaltime2017].sum(axis=1) > 0) & (nhts_agg_per_2017[Totaltime2017].sum(axis=1) < 1440)
]


In [342]:
nhts_agg_per_2017.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WK,AT_WKRMEET,AT_VOACT,...,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_HEAL,TT_RELIG,TT_SOMELSE,Total_TT,Total_AT,HW_TIME_2,HW_TIME_1
count,2.048650e+05,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,...,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000
mean,3.499740e+07,1.759744,24.584536,192.966451,9.500242,66.076665,5.457565,158.218349,4.552535,4.886930,...,3.529539,2.214141,3.388802,2.023367,1.555712,0.333322,88.183765,395.033422,956.782813,1062.961111
std,5.051717e+06,1.000001,38.991634,212.515163,7.720329,124.230944,47.406484,239.734678,35.912238,36.732264,...,16.794541,10.159077,15.340565,10.617908,8.282284,5.404447,80.451691,234.693025,253.316493,239.117793
min,3.000001e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,75.000000
25%,3.025985e+07,1.000000,10.000000,20.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.000000,180.000000,767.000000,867.000000
50%,3.052146e+07,1.000000,15.000000,85.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67.000000,431.000000,915.000000,1065.000000
75%,4.036686e+07,2.000000,30.000000,375.000000,14.000000,82.000000,0.000000,408.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,110.000000,572.000000,1170.000000,1282.000000
max,5.051557e+07,11.000000,1200.000000,1331.000000,97.000000,1260.000000,965.000000,1240.000000,1110.000000,1331.000000,...,735.000000,735.000000,660.000000,480.000000,420.000000,450.000000,1217.000000,1331.000000,1439.000000,1440.000000


In [343]:
result = nhts_agg_per_2017[nhts_agg_per_2017[Totaltime2017].sum(axis=1) == 1]
result

,uni_Per_ID,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WK,AT_WKRMEET,...,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_HEAL,TT_RELIG,TT_SOMELSE,Total_TT,Total_AT,HW_TIME_2,HW_TIME_1
13366,30067304_1,30067304,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440
17693,30089448_2,30089448,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440
25096,30127592_1,30127592,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440
26952,30136559_2,30136559,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440
26955,30136559_5,30136559,5,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440
53135,30269537_3,30269537,3,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440
57241,30290545_2,30290545,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440
68288,30346398_2,30346398,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440
70188,30355787_2,30355787,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440
81306,30412551_2,30412551,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440


In [344]:
# Define activity mapping for simpler aggregation
activity_map = {
    "ReguHMact": ["REGUHM"],
    "WorkFromHM": ["WKFHMP"],
    "WorkNonHM": ["WK", "WKRMEET", "VOACT"],
    "School": ["SCHSTU"],
    "Shopping": ["BUYGOOD"],
    "Social1": ["RECREAT"],
    "Social2": ["EXERS"],
    "Social3": ["VISFRI"],
    "Social4": ["RELIG"],
    "Personal_self": ["HEAL"],
    "Personal_meal": ["BUYMEAL"],
    "Family_business": ["CHILDC", "ADULTC", "BUYSER", "ERRAND", "SOMELSE"],
    "Pick_Drop": ["DOFPU", "CTYTRANS"]
}

# Loop through each activity group and compute aggregates
for activity, cols in activity_map.items():
    at_cols = [f"AT_{col}" for col in cols]
    tt_cols = [f"TT_{col}" for col in cols]

    nhts_agg_per_2017[f"AT_{activity}"] = nhts_agg_per_2017[at_cols].sum(axis=1)
    nhts_agg_per_2017[f"TT_{activity}"] = nhts_agg_per_2017[tt_cols].sum(axis=1)
    nhts_agg_per_2017[activity] = nhts_agg_per_2017[f"AT_{activity}"] + nhts_agg_per_2017[f"TT_{activity}"]


C:\Users\jinzh\AppData\Local\Temp\ipykernel_36784\4261005070.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts_agg_per_2017[f"AT_{activity}"] = nhts_agg_per_2017[at_cols].sum(axis=1)
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36784\4261005070.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts_agg_per_2017[f"TT_{activity}"] = nhts_agg_per_2017[tt_cols].sum(axis=1)
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36784\4261005070.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [345]:
nhts_agg_per_2017.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WK,AT_WKRMEET,AT_VOACT,...,Personal_self,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop
count,2.048650e+05,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,...,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000,204865.000000
mean,3.499740e+07,1.759744,24.584536,192.966451,9.500242,66.076665,5.457565,158.218349,4.552535,4.886930,...,8.033837,15.372894,5.857535,21.230430,9.529461,4.181768,13.711229,4.958456,6.385844,11.344300
std,5.051717e+06,1.000001,38.991634,212.515163,7.720329,124.230944,47.406484,239.734678,35.912238,36.732264,...,37.505671,36.239504,18.659967,47.875603,42.388522,14.691834,49.848804,25.939465,27.102443,45.188739
min,3.000001e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.025985e+07,1.000000,10.000000,20.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.052146e+07,1.000000,15.000000,85.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.036686e+07,2.000000,30.000000,375.000000,14.000000,82.000000,0.000000,408.000000,0.000000,0.000000,...,0.000000,8.000000,5.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.051557e+07,11.000000,1200.000000,1331.000000,97.000000,1260.000000,965.000000,1240.000000,1110.000000,1331.000000,...,975.000000,841.000000,840.000000,1170.000000,965.000000,800.000000,990.000000,1159.000000,986.000000,1365.000000


## Deal with Person

In [346]:
columns_per_2017 = [
    "HOUSEID", "PERSONID", "R_AGE", "R_HISP", "R_SEX", "R_RACE","EDUC",
    "WKRMHM",
    # Household character
    "HOMEOWN", "HHSIZE",#"LIF_CYC", "WRKCOUNT","NUMADLT",#"DELIVER",
    "HHVEHCNT", "HHFAMINC", "URBANSIZE", "URBRUR", #"CENSUS_R",
    # Mobility & Transportation
    #"DRIVER","WRKTRANS", "PTUSED","GCDWORK",
    # Travel day
    "TRAVDAY",
    #"YOUNGCHILD"
    #"MSASIZE", "CNTTDTR","MEDCOND","RIDESHARE",
    "DRVRCNT",
    #"RAIL",

]

nhts2017_per = nhts2017_per_1[columns_per_2017]

In [347]:
nhts2017_per["uni_Per_ID"]=nhts2017_per["HOUSEID"].astype(str) + "_" + nhts2017_per["PERSONID"].astype(str)

C:\Users\jinzh\AppData\Local\Temp\ipykernel_36784\1728817639.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2017_per["uni_Per_ID"]=nhts2017_per["HOUSEID"].astype(str) + "_" + nhts2017_per["PERSONID"].astype(str)


In [348]:
nhts2017_per = pd.merge(nhts2017_per,nhts2017_HH, on="HOUSEID", how="left")
nhts2017_per

,HOUSEID,PERSONID,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKRMHM,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,URBANSIZE,URBRUR,TRAVDAY,DRVRCNT,uni_Per_ID,YOUNGCHILD
0,30000007,1,67,2,2,2,3,-1,1,3,5,7,1,1,2,3,30000007_1,0
1,30000007,2,66,2,1,2,3,2,1,3,5,7,1,1,2,3,30000007_2,0
2,30000007,3,28,2,2,2,2,-1,1,3,5,7,1,1,2,3,30000007_3,0
3,30000008,1,55,2,1,1,5,2,1,2,4,8,6,2,5,2,30000008_1,0
4,30000008,2,49,2,2,1,4,2,1,2,4,8,6,2,5,2,30000008_2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264229,40794301,4,18,2,2,1,2,-1,1,7,5,6,6,2,2,4,40794301_4,0
264230,40794301,5,14,2,2,1,1,-1,1,7,5,6,6,2,2,4,40794301_5,0
264231,40794301,6,12,2,2,1,-1,-1,1,7,5,6,6,2,2,4,40794301_6,0
264232,40794301,7,8,2,2,1,-1,-1,1,7,5,6,6,2,2,4,40794301_7,0


In [349]:
#choose only workers
nhts2017_per = nhts2017_per[nhts2017_per["WKRMHM"].isin([1,2])]

In [350]:
nhts2017_per = nhts2017_per.replace([-9, -7, -8,-6,-5,-4,-3,-2,-1], np.nan)

In [351]:
nhts2017_per = nhts2017_per.dropna()

In [352]:
nhts2017_per.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103700 entries, 1 to 264233
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   HOUSEID     103700 non-null  int64  
 1   PERSONID    103700 non-null  int64  
 2   R_AGE       103700 non-null  float64
 3   R_HISP      103700 non-null  float64
 4   R_SEX       103700 non-null  float64
 5   R_RACE      103700 non-null  float64
 6   EDUC        103700 non-null  float64
 7   WKRMHM      103700 non-null  int64  
 8   HOMEOWN     103700 non-null  float64
 9   HHSIZE      103700 non-null  int64  
 10  HHVEHCNT    103700 non-null  int64  
 11  HHFAMINC    103700 non-null  float64
 12  URBANSIZE   103700 non-null  int64  
 13  URBRUR      103700 non-null  int64  
 14  TRAVDAY     103700 non-null  int64  
 15  DRVRCNT     103700 non-null  int64  
 16  uni_Per_ID  103700 non-null  object 
 17  YOUNGCHILD  103700 non-null  int64  
dtypes: float64(7), int64(10), object(1)
memory usage:

In [353]:
nhts2017_per.describe()

,HOUSEID,PERSONID,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKRMHM,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,URBANSIZE,URBRUR,TRAVDAY,DRVRCNT,YOUNGCHILD
count,1.037000e+05,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000,103700.000000
mean,3.499607e+07,1.574012,45.507406,1.908891,1.490106,3.688592,3.522739,1.827975,1.779267,2.691967,2.402575,6.898399,3.979171,1.207589,3.999248,2.074879,0.145188
std,5.051478e+06,0.745527,14.470419,0.287765,0.499905,14.806348,1.112325,0.377404,7.251781,1.311154,1.235657,2.396891,1.793947,0.405583,1.884519,0.842302,0.441705
min,3.000001e+07,1.000000,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,3.025886e+07,1.000000,34.000000,2.000000,1.000000,1.000000,3.000000,2.000000,1.000000,2.000000,2.000000,5.000000,2.000000,1.000000,2.000000,2.000000,0.000000
50%,3.052118e+07,1.000000,47.000000,2.000000,1.000000,1.000000,4.000000,2.000000,1.000000,2.000000,2.000000,7.000000,4.000000,1.000000,4.000000,2.000000,0.000000
75%,4.036600e+07,2.000000,57.000000,2.000000,2.000000,1.000000,4.000000,2.000000,1.000000,4.000000,3.000000,9.000000,6.000000,1.000000,6.000000,2.000000,0.000000
max,5.051557e+07,10.000000,92.000000,2.000000,2.000000,97.000000,5.000000,2.000000,97.000000,13.000000,12.000000,11.000000,6.000000,2.000000,7.000000,9.000000,5.000000


In [354]:
# 1: female; 0: otherwise
nhts2017_per["R_SEX"] = (nhts2017_per["R_SEX"].replace({1: 0, 2: 1}))
nhts2017_per["R_SEX"].describe()

count    103700.000000
mean          0.490106
std           0.499905
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: R_SEX, dtype: float64

In [355]:
#1: Urban Size with heavy rail or not 0:Otherwise
highway_map = {1: 0, 2: 0, 3: 0, 4: 0, 6: 0, 5: 1}
nhts2017_per["URBANSIZE"] = nhts2017_per["URBANSIZE"].map(highway_map)
print(nhts2017_per["URBANSIZE"].describe())

count    103700.000000
mean          0.128293
std           0.334418
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: URBANSIZE, dtype: float64


In [356]:
# 1: urban 0: Rural
nhts2017_per["URBRUR"] = (nhts2017_per["URBRUR"].replace({2: 0}))
nhts2017_per["URBRUR"].describe()

count    103700.000000
mean          0.792411
std           0.405583
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: URBRUR, dtype: float64

In [357]:
# 1: work from home 0: otherwise
wkhw = {2:0, 1:1}
nhts2017_per["WKRMHM"] = nhts2017_per["WKRMHM"].map(wkhw)
nhts2017_per["WKRMHM"].describe()

count    103700.000000
mean          0.172025
std           0.377404
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: WKRMHM, dtype: float64

In [358]:
# 1: education level higher than highschool  0:otherwise
edu_mapping = {1: 0, 2: 1, 3: 1, 4: 1, 5: 1}
nhts2017_per["Highschool"] = nhts2017_per["EDUC"].map(edu_mapping)
print(nhts2017_per["Highschool"].describe())

count    103700.000000
mean          0.969749
std           0.171277
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: Highschool, dtype: float64


In [359]:
# 1: education level higher than college  0:otherwise
edu_college = {1: 0, 2: 0, 3: 0, 4: 1, 5: 1}
nhts2017_per["EDU_College"] = nhts2017_per["EDUC"].map(edu_college)
print(nhts2017_per["EDU_College"].describe())

count    103700.000000
mean          0.510154
std           0.499899
min           0.000000
25%           0.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: EDU_College, dtype: float64


In [360]:
hmown = {2: 0, 97: 0, 1: 1}
nhts2017_per["HM_OWN"] = nhts2017_per["HOMEOWN"].map(hmown)
print(nhts2017_per["HM_OWN"].describe())

count    103700.000000
mean          0.766731
std           0.422914
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: HM_OWN, dtype: float64


In [361]:
nhts2017_per["R_HISP"] = (nhts2017_per["R_HISP"].replace({2: 0, 1: 1}))
nhts2017_per["R_HISP"].describe()

count    103700.000000
mean          0.091109
std           0.287765
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: R_HISP, dtype: float64

In [362]:
nhts2017_per["NONHISWHITE"] = np.where(
    (nhts2017_per["R_HISP"] == 0) & (nhts2017_per["R_RACE"] == 1),
    1,
    0
)
nhts2017_per["NONHISWHITE"].describe()

count    103700.000000
mean          0.757146
std           0.428810
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: NONHISWHITE, dtype: float64

In [363]:
# Create binary variable for Race_Black: 1 if Black, else 0
race_black_map = {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 97: 0}
nhts2017_per["Race_Black"] = nhts2017_per["R_RACE"].map(race_black_map)
nhts2017_per["Race_Black"].describe()

count    103700.000000
mean          0.070820
std           0.256525
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: Race_Black, dtype: float64

In [364]:
nhts2017_per["Race_Asian"] = nhts2017_per["R_RACE"].replace({
    1: 0,  # White
    2: 0,  # Black
    3: 1,  # Asian
    4: 0,  # American Indian or Alaska Native
    5: 0,  # Pacific Islander
    6: 0,  # Other
    97: 0  # Other
})

# Check results:
nhts2017_per["Race_Asian"].value_counts()

Race_Asian
0.0    98533
1.0     5167
Name: count, dtype: int64

In [365]:
nhts2017_per["Race_White"] = nhts2017_per["R_RACE"].replace({
    1: 1,  # White
    2: 0,  # Black
    3: 0,  # Asian
    4: 0,  # American Indian or Alaska Native
    5: 0,  # Pacific Islander
    6: 0,  # Other
    97: 0  # Other
})

# Check results:
nhts2017_per["Race_White"].value_counts()

Race_White
1.0    84815
0.0    18885
Name: count, dtype: int64

In [366]:
income_map = {
    1: 7.5,
    2: 12.5,
    3: 20.0,
    4: 30.0,
    5: 42.5,
    6: 62.5,
    7: 87.5,
    8: 112.5,
    9: 137.5,
    10: 175.0,
    11: 250.0
}
nhts2017_per["HHFAMINC"] = nhts2017_per["HHFAMINC"].map(income_map)

In [367]:
# Weekday: Monday through Friday (02-06)
weekday_map = {1: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 0}
nhts2017_per["weekday"] = nhts2017_per["TRAVDAY"].map(weekday_map)

In [368]:
nhts2017_per

,HOUSEID,PERSONID,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKRMHM,HOMEOWN,HHSIZE,...,uni_Per_ID,YOUNGCHILD,Highschool,EDU_College,HM_OWN,NONHISWHITE,Race_Black,Race_Asian,Race_White,weekday
1,30000007,2,66.0,0.0,0.0,2.0,3.0,0,1.0,3,...,30000007_2,0,1,0,1,0,1,0.0,0.0,1
3,30000008,1,55.0,0.0,0.0,1.0,5.0,0,1.0,2,...,30000008_1,0,1,1,1,1,0,0.0,1.0,1
4,30000008,2,49.0,0.0,1.0,1.0,4.0,0,1.0,2,...,30000008_2,0,1,1,1,1,0,0.0,1.0,1
5,30000012,1,45.0,0.0,1.0,1.0,5.0,0,1.0,1,...,30000012_1,0,1,1,1,1,0,0.0,1.0,1
10,30000039,1,82.0,0.0,0.0,1.0,5.0,0,1.0,2,...,30000039_1,0,1,1,1,1,0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264213,40794238,2,40.0,0.0,1.0,1.0,3.0,0,2.0,3,...,40794238_2,0,1,0,0,1,0,0.0,1.0,1
264216,40794241,2,39.0,0.0,0.0,1.0,4.0,0,1.0,2,...,40794241_2,0,1,1,1,1,0,0.0,1.0,1
264218,40794260,1,35.0,0.0,1.0,1.0,4.0,0,1.0,4,...,40794260_1,1,1,1,1,1,0,0.0,1.0,1
264227,40794301,2,59.0,0.0,0.0,1.0,4.0,0,1.0,7,...,40794301_2,0,1,1,1,1,0,0.0,1.0,1


## Combine person and Trip

In [369]:
nhts2017_per_trip = pd.merge(nhts2017_per, nhts_agg_per_2017, on="uni_Per_ID", how="inner")

In [370]:
nhts2017_per_trip

,HOUSEID_x,PERSONID_x,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKRMHM,HOMEOWN,HHSIZE,...,Personal_self,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop
0,30000008,1,55.0,0.0,0.0,1.0,5.0,0,1.0,2,...,0,0,0,0,0,0,0,0,0,0
1,30000012,1,45.0,0.0,1.0,1.0,5.0,0,1.0,1,...,0,0,0,0,0,0,0,0,0,0
2,30000039,1,82.0,0.0,0.0,1.0,5.0,0,1.0,2,...,0,70,5,75,0,0,0,0,0,0
3,30000041,1,44.0,0.0,0.0,1.0,5.0,0,1.0,2,...,0,0,0,0,0,0,0,0,0,0
4,30000041,2,40.0,0.0,1.0,1.0,1.0,0,1.0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89515,40794238,2,40.0,0.0,1.0,1.0,3.0,0,2.0,3,...,0,0,0,0,0,0,0,0,0,0
89516,40794241,2,39.0,0.0,0.0,1.0,4.0,0,1.0,2,...,71,87,8,95,0,0,0,0,0,0
89517,40794260,1,35.0,0.0,1.0,1.0,4.0,0,1.0,4,...,0,88,22,110,0,0,0,5,38,43
89518,40794301,2,59.0,0.0,0.0,1.0,4.0,0,1.0,7,...,0,28,10,38,0,0,0,0,0,0


In [371]:
print(nhts2017_per_trip.columns)


Index(['HOUSEID_x', 'PERSONID_x', 'R_AGE', 'R_HISP', 'R_SEX', 'R_RACE', 'EDUC',
       'WKRMHM', 'HOMEOWN', 'HHSIZE',
       ...
       'Personal_self', 'AT_Personal_meal', 'TT_Personal_meal',
       'Personal_meal', 'AT_Family_business', 'TT_Family_business',
       'Family_business', 'AT_Pick_Drop', 'TT_Pick_Drop', 'Pick_Drop'],
      dtype='object', length=114)


In [372]:
# For Teleworkers, their HW_TIME need to + work from home time

#nhts2017_per_trip["HW_TIME"] = np.where( 
#    nhts2017_per_trip["WKRMHM"] == 1,
#    nhts2017_per_trip["HW_TIME_1"] + nhts2017_per_trip ["ActivityatHM_1"],
#    nhts2017_per_trip["HW_TIME_1"]
#)
nhts2017_per_trip["HW_TIME"] = nhts2017_per_trip["HW_TIME_1"]

In [373]:
nhts2017_per_trip

,HOUSEID_x,PERSONID_x,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKRMHM,HOMEOWN,HHSIZE,...,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop,HW_TIME
0,30000008,1,55.0,0.0,0.0,1.0,5.0,0,1.0,2,...,0,0,0,0,0,0,0,0,0,705
1,30000012,1,45.0,0.0,1.0,1.0,5.0,0,1.0,1,...,0,0,0,0,0,0,0,0,0,725
2,30000039,1,82.0,0.0,0.0,1.0,5.0,0,1.0,2,...,70,5,75,0,0,0,0,0,0,1110
3,30000041,1,44.0,0.0,0.0,1.0,5.0,0,1.0,2,...,0,0,0,0,0,0,0,0,0,840
4,30000041,2,40.0,0.0,1.0,1.0,1.0,0,1.0,2,...,0,0,0,0,0,0,0,0,0,810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89515,40794238,2,40.0,0.0,1.0,1.0,3.0,0,2.0,3,...,0,0,0,0,0,0,0,0,0,840
89516,40794241,2,39.0,0.0,0.0,1.0,4.0,0,1.0,2,...,87,8,95,0,0,0,0,0,0,1248
89517,40794260,1,35.0,0.0,1.0,1.0,4.0,0,1.0,4,...,88,22,110,0,0,0,5,38,43,627
89518,40794301,2,59.0,0.0,0.0,1.0,4.0,0,1.0,7,...,28,10,38,0,0,0,0,0,0,878


### filter the person whose travel time is no more than 6 hours per day

In [374]:
# filter rows of nhts2017_per_trip that have uni_Per_ID matching regular_TT_2017
nhts2017_per_trip = nhts2017_per_trip[
    nhts2017_per_trip["uni_Per_ID"].isin(regular_TT_2017["uni_Per_ID"].unique())
]

#supercommute_TT_2017 
nhts2017_per_trip 

,HOUSEID_x,PERSONID_x,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKRMHM,HOMEOWN,HHSIZE,...,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop,HW_TIME
0,30000008,1,55.0,0.0,0.0,1.0,5.0,0,1.0,2,...,0,0,0,0,0,0,0,0,0,705
1,30000012,1,45.0,0.0,1.0,1.0,5.0,0,1.0,1,...,0,0,0,0,0,0,0,0,0,725
2,30000039,1,82.0,0.0,0.0,1.0,5.0,0,1.0,2,...,70,5,75,0,0,0,0,0,0,1110
3,30000041,1,44.0,0.0,0.0,1.0,5.0,0,1.0,2,...,0,0,0,0,0,0,0,0,0,840
4,30000041,2,40.0,0.0,1.0,1.0,1.0,0,1.0,2,...,0,0,0,0,0,0,0,0,0,810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89515,40794238,2,40.0,0.0,1.0,1.0,3.0,0,2.0,3,...,0,0,0,0,0,0,0,0,0,840
89516,40794241,2,39.0,0.0,0.0,1.0,4.0,0,1.0,2,...,87,8,95,0,0,0,0,0,0,1248
89517,40794260,1,35.0,0.0,1.0,1.0,4.0,0,1.0,4,...,88,22,110,0,0,0,5,38,43,627
89518,40794301,2,59.0,0.0,0.0,1.0,4.0,0,1.0,7,...,28,10,38,0,0,0,0,0,0,878


In [375]:
nhts2017_per_trip.describe()

,HOUSEID_x,PERSONID_x,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKRMHM,HOMEOWN,HHSIZE,...,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop,HW_TIME
count,8.832200e+04,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,...,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000
mean,3.497910e+07,1.562068,45.587577,0.090034,0.494758,3.620072,3.541202,0.171735,1.782704,2.687383,...,13.778005,4.971038,18.749043,5.354430,2.751534,8.105965,4.292849,5.587815,9.880664,944.035257
std,5.050581e+06,0.735655,14.445928,0.286232,0.499975,14.609376,1.112029,0.377152,7.289877,1.304394,...,34.427468,13.561259,43.352748,27.338158,10.465026,33.577301,22.972556,18.224327,35.170952,223.270237
min,3.000001e+07,1.000000,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000
25%,3.025767e+07,1.000000,34.000000,0.000000,0.000000,1.000000,3.000000,0.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,795.000000
50%,3.051977e+07,1.000000,47.000000,0.000000,0.000000,1.000000,4.000000,0.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,885.000000
75%,4.036470e+07,2.000000,57.000000,0.000000,1.000000,1.000000,4.000000,0.000000,1.000000,4.000000,...,5.000000,4.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1090.000000
max,5.051557e+07,10.000000,92.000000,1.000000,1.000000,97.000000,5.000000,1.000000,97.000000,13.000000,...,803.000000,330.000000,867.000000,872.000000,315.000000,890.000000,835.000000,320.000000,855.000000,1440.000000


In [376]:
column_list2017 = nhts2017_per_trip.columns.tolist()
print(column_list2017)

['HOUSEID_x', 'PERSONID_x', 'R_AGE', 'R_HISP', 'R_SEX', 'R_RACE', 'EDUC', 'WKRMHM', 'HOMEOWN', 'HHSIZE', 'HHVEHCNT', 'HHFAMINC', 'URBANSIZE', 'URBRUR', 'TRAVDAY', 'DRVRCNT', 'uni_Per_ID', 'YOUNGCHILD', 'Highschool', 'EDU_College', 'HM_OWN', 'NONHISWHITE', 'Race_Black', 'Race_Asian', 'Race_White', 'weekday', 'HOUSEID_y', 'PERSONID_y', 'TRVLCMIN', 'DWELTIME', 'WHYTO', 'AT_REGUHM', 'AT_WKFHMP', 'AT_WK', 'AT_WKRMEET', 'AT_VOACT', 'AT_DOFPU', 'AT_CTYTRANS', 'AT_SCHSTU', 'AT_CHILDC', 'AT_ADULTC', 'AT_BUYGOOD', 'AT_BUYSER', 'AT_BUYMEAL', 'AT_ERRAND', 'AT_RECREAT', 'AT_EXERS', 'AT_VISFRI', 'AT_HEAL', 'AT_RELIG', 'AT_SOMELSE', 'TT_REGUHM', 'TT_WKFHMP', 'TT_WK', 'TT_WKRMEET', 'TT_VOACT', 'TT_DOFPU', 'TT_CTYTRANS', 'TT_SCHSTU', 'TT_CHILDC', 'TT_ADULTC', 'TT_BUYGOOD', 'TT_BUYSER', 'TT_BUYMEAL', 'TT_ERRAND', 'TT_RECREAT', 'TT_EXERS', 'TT_VISFRI', 'TT_HEAL', 'TT_RELIG', 'TT_SOMELSE', 'Total_TT', 'Total_AT', 'HW_TIME_2', 'HW_TIME_1', 'AT_ReguHMact', 'TT_ReguHMact', 'ReguHMact', 'AT_WorkFromHM', 'TT_W

In [377]:
export2017 = [
    'HOUSEID_x', 'PERSONID_x', 'R_AGE', 'R_HISP', 'R_SEX', 'R_RACE', 'WKRMHM',
    'HHSIZE', 'HHVEHCNT', 'HHFAMINC', 'URBANSIZE', 'URBRUR', 'uni_Per_ID',
    'YOUNGCHILD', 'Highschool', 'EDU_College', 'HM_OWN', 'NONHISWHITE', 
    'Race_Black', 'Race_Asian', 'Race_White',
    #added
    #"single_adult","multi_adult",
    #"driver",
    #"active_trans", "priv_veh", "small_motor", "transit",
    #"weekend", 
    "weekday",
    #"WRKCOUNT", "PTUSED","GCDWORK", "NUMADLT","DELIVER",
    #"disability", 
    #"small_msa", "large_msa",
    #"CNTTDTR","RIDESHARE",
    "DRVRCNT",
    #census
    #"northeast","midwest","south","west",
    #"has_rail",
    #Household Income
    #"low_income","lower_mid_income", "middle_income", "upper_mid_income", "high_income",
    # urban size
    #"large_urban_rail","large_urban_norail","medium_urban","small_urban",

    #time use
    'Total_TT', 'Total_AT', 
    'AT_ReguHMact', 'TT_ReguHMact', 'ReguHMact', 'AT_WorkFromHM', 
    'TT_WorkFromHM', 'WorkFromHM', 'AT_WorkNonHM', 'TT_WorkNonHM', 
    'WorkNonHM', 'AT_School', 'TT_School', 'School', 'AT_Shopping', 
    'TT_Shopping', 'Shopping', 'AT_Social1', 'TT_Social1', 'Social1', 
    'AT_Social2', 'TT_Social2', 'Social2', 'AT_Social3', 'TT_Social3', 
    'Social3', 'AT_Social4', 'TT_Social4', 'Social4', 'AT_Personal_self', 
    'TT_Personal_self', 'Personal_self', 'AT_Personal_meal', 
    'TT_Personal_meal', 'Personal_meal', 'AT_Family_business', 
    'TT_Family_business', 'Family_business', 'AT_Pick_Drop', 
    'TT_Pick_Drop', 'Pick_Drop', 'HW_TIME'
    
]


In [378]:
out_2017 = PROCESSED_DATA_DIR / "nhts2017_per_trip.csv"
nhts2017_per_trip[export2017].to_csv(out_2017, index=False)
print("Saved:", out_2017)

Saved: C:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report\data\processed\nhts2017_per_trip.csv


In [379]:
nhts2017_per_trip["boomers2"] = np.where(
    nhts2017_per_trip["R_AGE"].isin(list(range(53, 63))),
    1,
    0
)

nhts2017_per_trip["genX"] = np.where(
    nhts2017_per_trip["R_AGE"].isin(list(range(37, 53))),
    1,
    0
)

nhts2017_per_trip["Millennials"] = np.where(
    nhts2017_per_trip["R_AGE"].isin(list(range(21, 37))),
    1,
    0
)

nhts2017_per_trip["genZ"] = np.where(
    nhts2017_per_trip["R_AGE"].isin(list(range(5, 21))),
    1,
    0
)

C:\Users\jinzh\AppData\Local\Temp\ipykernel_36784\3318709579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2017_per_trip["boomers2"] = np.where(
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36784\3318709579.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2017_per_trip["genX"] = np.where(
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36784\3318709579.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [380]:
outdoor_time_2017 = ["School", "Shopping", "Social1", "Social2", 
    "Social3", "Social4", "Personal_self", 
    "Personal_meal", "Family_business", "Pick_Drop"]

for tcate in outdoor_time_2017:
    log10_var = nhts2017_per_trip[tcate] /nhts2017_per_trip["HW_TIME"]
    ln10_var1 = log10_var + 1
    nhts2017_per_trip[f"ln_{tcate}"] = np.log10(ln10_var1)
    #nhts2017_per_trip[f"ln_{tcate}"] = np.arcsinh(log10_var)

ln_outdoor_time_2017 = ["ln_School", "ln_Shopping", "ln_Social1", "ln_Social2", 
                        "ln_Social3", "ln_Social4", "ln_Personal_self", 
                        "ln_Personal_meal", "ln_Family_business", "ln_Pick_Drop"]
nhts2017_per_trip[ln_outdoor_time_2017].describe()

C:\Users\jinzh\AppData\Local\Temp\ipykernel_36784\735680283.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2017_per_trip[f"ln_{tcate}"] = np.log10(ln10_var1)
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36784\735680283.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2017_per_trip[f"ln_{tcate}"] = np.log10(ln10_var1)
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36784\735680283.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,ln_School,ln_Shopping,ln_Social1,ln_Social2,ln_Social3,ln_Social4,ln_Personal_self,ln_Personal_meal,ln_Family_business,ln_Pick_Drop
count,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000,88322.000000
mean,0.003350,0.009555,0.007738,0.004745,0.008003,0.003526,0.001780,0.008786,0.003583,0.004746
std,0.024773,0.020052,0.030863,0.017205,0.031534,0.018102,0.011372,0.020805,0.014825,0.016698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.012374,0.000000,0.000000,0.000000,0.000000,0.000000,0.005830,0.000000,0.000000
max,0.404391,0.477121,0.683134,0.515061,0.866106,0.629162,0.490910,0.444683,0.419895,0.470781


In [381]:
sns.histplot(data=nhts2017_per_trip, x="ln_Social1")
fig_path = FIGURES_DIR / "hist_ln_Social1_2017.png"
plt.savefig(fig_path, dpi=200, bbox_inches="tight")
plt.close()
print("Saved:", fig_path)

Saved: C:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report\outputs\figures\hist_ln_Social1_2017.png


In [382]:
ind_vars_2017 = [
    "R_AGE", "R_HISP", "R_SEX", "R_RACE", "HHSIZE", "HHVEHCNT", "HHFAMINC",
    "URBANSIZE", "URBRUR", "YOUNGCHILD", "Highschool", "EDU_College","WKRMHM",
    "HM_OWN", "NONHISWHITE", "Race_Black", "Race_White","Race_Asian",
    #added
    #"single_adult",#"multi_adult",
   #"driver",
   # "active_trans",  "small_motor", "transit",#"priv_veh",
    "weekday",
   # "WRKCOUNT", "PTUSED","GCDWORK","NUMADLT","DELIVER",
    #"disability", 
   # "small_msa", "large_msa", 
    #"has_rail",
    #"CNTTDTR","RIDESHARE",
    "DRVRCNT",
    #census
   # "northeast","midwest","south","west",
        #Household Income
   # "low_income","lower_mid_income", "middle_income", "upper_mid_income", "high_income",
    # urban size
    #"large_urban_rail","large_urban_norail","medium_urban","small_urban",
    #gen
    "genZ", "Millennials", "genX", "boomers2"
]
dep_vars_2017 = ["ln_School", "ln_Shopping", "ln_Social1", "ln_Social2", 
            "ln_Social3", "ln_Social4", "ln_Personal_self", 
            "ln_Personal_meal", "ln_Family_business", "ln_Pick_Drop"]

In [383]:
plt.figure(figsize=(12, 10))

X = nhts2017_per_trip[ind_vars_2017]
sns.heatmap(X.corr(), annot=True)

fig_path = FIGURES_DIR / "heatmap_ind_vars_2017.png"

plt.savefig(fig_path, dpi=200, bbox_inches="tight")

plt.close()

print("Saved figure to:", fig_path)

Saved figure to: C:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report\outputs\figures\heatmap_ind_vars_2017.png


In [384]:
plt.figure(figsize=(12, 10))

X = nhts2017_per_trip[dep_vars_2017]
sns.heatmap(X.corr(), annot=True)

fig_path = FIGURES_DIR / "heatmap_dep_vars_2017.png"

plt.savefig(fig_path, dpi=200, bbox_inches="tight")

plt.close()

print("Saved figure to:", fig_path)

Saved figure to: C:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report\outputs\figures\heatmap_dep_vars_2017.png


In [385]:
#Check VIF
X = nhts2017_per_trip[ind_vars_2017].dropna()
vif_data = pd.DataFrame({
    "variable": X.columns,
    "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
})
print(vif_data)

       variable        VIF
0         R_AGE  36.242693
1        R_HISP   8.029289
2         R_SEX   2.017723
3        R_RACE   2.869980
4        HHSIZE  14.151051
5      HHVEHCNT   8.948504
6      HHFAMINC   4.751864
7     URBANSIZE   1.264105
8        URBRUR   5.430705
9    YOUNGCHILD   1.667152
10   Highschool  36.191570
11  EDU_College   2.704611
12       WKRMHM   1.319718
13       HM_OWN   5.619734
14  NONHISWHITE  81.507485
15   Race_Black   2.999681
16   Race_White  89.103543
17   Race_Asian   2.482520
18      weekday   4.954145
19      DRVRCNT  20.583366
20         genZ   2.158118
21  Millennials   8.008287
22         genX   5.472491
23     boomers2   3.070273


In [386]:
nhts2017_per_trip = nhts2017_per_trip[export2017 + ln_outdoor_time_2017 + ["genZ", "Millennials", "genX", "boomers2"]]

In [387]:
nhts2017_per_trip.shape

(88322, 79)

In [388]:
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

out_2017 = PROCESSED_DATA_DIR / "nhts2017_per_trip.csv"
nhts2017_per_trip.to_csv(out_2017, index=False)

print("Saved 2017 processed data to:", out_2017)

Saved 2017 processed data to: C:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report\data\processed\nhts2017_per_trip.csv
